In [1]:
# importing the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime as dt

In [2]:
# loading the data frame and converting attiributes to object type or datetime.
listings = pd.read_csv('../data/listings_clean.csv')
# converting data type to datetime.
listings.drop(['host_id','host_since'],axis=1,inplace=True)
# changing data type of is,host_id columns to object
listings.select_dtypes('object').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6882 entries, 0 to 6881
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   neighbourhood  6882 non-null   object
 1   property_type  6882 non-null   object
dtypes: object(2)
memory usage: 107.7+ KB


**Summary:**

There are 2 columns that are in object type. I will do dummy transformation for these columns. However, previously I have done following as preprocessing during data wrangling:

1. Did ordinal encoder for the host_response_time. 
2. Processed the host_verifications column and did ordinal encoding.
3. Dropped the neighbourhood column since it encodes for large cities/areas that would not provide detailed information.
4. Renamed neighbourhood_cleansed column as neighbourhood and will do dummy transformation using pd.get_dummies().
5. Dropped the property_type column since it contains subgroups of room_type such as entire_home_bungalow etc. Renamed room_type as property type and will use pd.get_dummies(). 
6. Amenities column was converted a list and in turn a set of all amenities in the data frame. Created new columns by encoding amenities for their presence in the listinf. Details are in data wrangling notebook.
7. host_since column was converted to date time and created a new column as host_days_active by subtracting host_since from 2020_03_02. host_since column is dropped.
8. Repeated same process for first_review and last_review columns. 
9. Replaced missing values with median value of the columns

In [3]:
transformed_df = pd.get_dummies(listings)

In [4]:
transformed_df.to_csv(r'../data/listings_premodel.csv', index = False, header=True)